# SQL Operations Demo

One of the benefits of Spark is its support for SQL operations when it comes to streaming data. 

To do this, one first creates a `SparkSession` using the `SparkContext` that the `StreamingContext` is using. 


### Demo

For this demonstration, we will be using Spark and SQL to analyze data from a DStream.

In [22]:
import findspark
# TODO: your path will likely not have 'jilg' in it. Change it to reflect your path.
findspark.init('/home/jilg/Downloads/spark-3.3.2-bin-hadoop3')

In [23]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from operator import add
from pyspark.sql.functions import regexp_replace, col, trim, lower, desc
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql import Row, SparkSession
import re
import sys

In [24]:
#sc = SparkContext(master="local[2]", appName="PysparkSQLNetworkWordCount")
sc = SparkContext.getOrCreate()



ssc = StreamingContext(sc, 1)
sqlContext = SQLContext(sc)

In [25]:
lines = ssc.textFileStream("/home/jilg/python-spark-streaming/2_basics/data/")
words = lines.flatMap(lambda line: re.split(' ', line.lower().strip()))

In [26]:
# The section below is for converting RDDs of the words DStream to DataFrame and run SQL query
def analyze(time, rdd):
    print("========= %s =========" % str(time))
    try:
        # words of more than three characters
        rdd = rdd.filter(lambda x: len(x)>3)
        rdd.collect()

        # 1 count per word
        rdd = rdd.map(lambda w:(w,1))
        words_df = sqlContext.createDataFrame(rdd,['word','count'])
        words_df.show()

        # replace punctuation
        df_transformed = words_df.select(lower(trim(regexp_replace(col('word'),r'[.,\/#$%^&*()-_+=~!"\s]*',''))).alias('keywords'))

        # Print the top 250 words
        top_words = sqlContext.createDataFrame(df_transformed.groupby('keywords').count().sort(desc('count')).take(251))
        top_words.pprint()
    except:
        pass
words.foreachRDD(analyze)

In [27]:
ssc.start()

========= 2023-02-19 12:18:18 =========
========= 2023-02-19 12:18:19 =========
========= 2023-02-19 12:18:20 =========
========= 2023-02-19 12:18:21 =========
========= 2023-02-19 12:18:22 =========
========= 2023-02-19 12:18:23 =========
========= 2023-02-19 12:18:24 =========
========= 2023-02-19 12:18:25 =========
========= 2023-02-19 12:18:26 =========
========= 2023-02-19 12:18:27 =========
========= 2023-02-19 12:18:28 =========
========= 2023-02-19 12:18:29 =========


In [28]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)

========= 2023-02-19 12:18:30 =========
========= 2023-02-19 12:18:31 =========


## References
1. https://spark.apache.org/docs/latest/streaming-programming-guide.html#dataframe-and-sql-operations
2. https://spark.apache.org/docs/latest/sql-programming-guide.html